<a href="https://colab.research.google.com/github/tnystul/DSCB225/blob/main/Code/Image_Analysis_Problem_Set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import packages, functions, and data

In [1]:
# import python packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import skimage
import plotly.express as px

In [2]:
# define functions

def subtract_background(image, background):
    dtype = image.dtype
    return np.clip(image.astype(np.int16) - background, 0, 65535).astype(dtype)

def random_cmap():
    np.random.seed(42)
    cmap = matplotlib.colors.ListedColormap (np.random.rand(256,4))
    # value 0 should just be transparent
    cmap.colors[:,3] = 0.5
    cmap.colors[0,:] = 1
    cmap.colors[0,3] = 0

    # if image is a mask, color (last value) should be red
    cmap.colors[-1,0] = 1
    cmap.colors[-1,1:3] = 0
    return cmap

In [3]:
%%capture
!pip install -U tifffile
!pip install -U aicsimageio

from aicsimageio import AICSImage
!pip install aicspylibczi>=3.1.1 fsspec>=2022.8.0

In [4]:
!wget https://ucsf.box.com/shared/static/3ij7e92913368qargvrjjxg704ceo4sk.tif -O image.tif

--2025-03-15 23:12:55--  https://ucsf.box.com/shared/static/3ij7e92913368qargvrjjxg704ceo4sk.tif
Resolving ucsf.box.com (ucsf.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to ucsf.box.com (ucsf.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/3ij7e92913368qargvrjjxg704ceo4sk.tif [following]
--2025-03-15 23:12:56--  https://ucsf.box.com/public/static/3ij7e92913368qargvrjjxg704ceo4sk.tif
Reusing existing connection to ucsf.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ucsf.app.box.com/public/static/3ij7e92913368qargvrjjxg704ceo4sk.tif [following]
--2025-03-15 23:12:57--  https://ucsf.app.box.com/public/static/3ij7e92913368qargvrjjxg704ceo4sk.tif
Resolving ucsf.app.box.com (ucsf.app.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to ucsf.app.box.com (ucsf.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response..

In [5]:
data = AICSImage('/content/image.tif')
data.shape

(1, 3, 1, 241, 349)

In [6]:
img = data.get_image_dask_data('CYX', c=1)
img.shape

(3, 241, 349)

# Segment the DAPI channel and count the number of cells in the image

In [7]:
# View the three channels using the plt.subplots function



In [8]:
 # plot them again, this time adding titles for the Golgi, nuclei, and cytoplasm channels



Use the px.histogram function to plot a histogram of the intensity values in
the Nuclei channel

Based on the results of the histogram, plot 4 background subtraction options

Pick the one that looks best (in my answer, they all look pretty similar) and use that value to generate an image with the background subtracted from the Nuclei channel

Use the otsu filter to calculate a threshold for the Nuclei channel

Generate a mask of the Nuclei channel using the otsu value.  Then, use the plt function to view the image and the mask side-by-side

In [9]:
def generate_template(radius):
  template = np.zeros((60,60))
  center = [(template.shape[0]-1)/2,(template.shape[1]-1)/2]
  Y, X = np.mgrid[0:template.shape[0],0:template.shape[1]]
  dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
  template[dist_from_center<=radius] = 1
  return(template)

Using the function above, write a for loop that will sample radii ranging from 10-18 and plot the results in a 3 x 3 grid.

*   Use ax = fig.add_subplot(3,3,x+1) to set up the subplots
*   Use ax.imshow to show the Nuclei
*   Use ax.scatter to show the local max indices
*   Use ax.title.set_text to title each subplot with the radius value



Select the radius value you like the most and use it to

*   Generate an array of local maxima called local_max_indicies
*   Generate a boolean array of the local maxima
*   Eliminate the points that do not overlap with the image mask
*   Get the peak coordinates
*   Plot the peaks on top of the Nuclear channel of the image

Segment the image by

*   Generating labels from the masked peaks
*   Running the watershed algorithm, using img_p, the masked peaks, and the img_mask as inputs
*   define cmap using the random_cmap function and then plot the Nuclei and watershed labels together



Does that look right?  Try rerunning the watershed function again but this time, add in compactness = 3 as an additional parameter and plot the image and labels again.  What changed?

Generate a regionprops table and print its shape.  How many nuclei are segmented in this image?

# Segment the cytoplasm channel and quantify the mean signal intensity per cell

Now, generate a cytoplasm mask performing the following steps:

*   Create a "watershed mask" of the Cytoplasm channel using the same labels you generated from the masked peaks and the watershed segmentation tool (set compactness = 1)
*   Create a "foreground mask" that identifies all the pixels with an intensity above 20 in the Cytoplasm channel.  
*   Remove the parts of the watershed mask that **do** overlap the nuclear masks, so the nuclear signal doesn't contribute to the nuclear signal
*   Remove the parts of the watershed mask that **do not** overlap with the foreground mask

Note, although masks are boolean (T/F) arrays, they can be used in equations and python interprets True as 1 and False as 0

Display the results in a figure with 3 subplots:  one showing the cytoplasm signal, one showing the cytoplasm mask, and one showing both of these images merged together

Measure the mean intensity of the cytoplasm channel and store the results as a Pandas DataFrame

Create a histogram showing the mean intensity values in each cell